In [30]:
import os
import csv
import MySQLdb
import EmailCreateSend
pathName = os.getcwd()
index = []   #track image IDs 
logFiles = []
fileNames = os.listdir(pathName)

##Load all csv files in directory
##Script can scan multiple csv logs 
for fileNames in fileNames:
    if fileNames.endswith(".csv"):
        logFiles.append(fileNames)
        
##Loop for each log file
for i in logFiles:
    file = open(os.path.join(pathName, i), newline='')
    reader = csv.reader(file, delimiter=',')
    print('\n'+i)   ##print log file name 
    
    for row in reader:
        content = list(row[i] for i in [0,1]) ##[0,1] are included columns from CNN csv logs

        print (content)
        if float(content[1]) > .5:
            print('Violation')  
            index.append(content[0])
            print(index)


log-03_06_2019.csv
['1', '0.9']
Violation
['1']
['2', '0.9']
Violation
['1', '2']
['3', '0']
['4', '0.051306013']
['5', '0']
['6', '0.022467662']


In [31]:
db = MySQLdb.connect('remotemysql.com', '5i48ibmWpC', 'Wk4CcxeIyW', '5i48ibmWpC')
cursor = db.cursor()

In [32]:
## TODO: Grab info from database and add to email
#query = "SELECT U.USER_ID, U.FNAME, U.LNAME, SS.SCREENSHOT_PATH, SS.URL FROM USERS U JOIN SCREENSHOTS SS ON U.USER_ID = SS.SS_USER WHERE SS.SS_ID={}".format(index[0]) 
query = "SELECT U.USER_ID, U.FNAME, U.LNAME, SS.SCREENSHOT_PATH, SS.URL FROM USERS U JOIN SCREENSHOTS SS ON U.USER_ID = SS.SS_USER WHERE SS.SS_ID IN (%s)" % ",".join(map(str,index))
cursor.execute(query)
results = cursor.fetchall()

In [33]:
EmailCreateSend.send(results)